In [175]:
import geopandas as gpd
import pandas as pd
from os import listdir
import seaborn as sns
from shapely.geometry import Polygon
import geoparquet_io as gpio
import numpy as np

In [131]:
size = 1000

In [132]:
liste_files = listdir('carreaux/')

In [133]:
liste_dep = pd.read_json('dep_dispo_2026_01_07.json')
liste_dep.head(3)

,codgeo,year,link
168,95,2021,https://data.geopf.fr/telechargement/download/...
165,94,2021,https://data.geopf.fr/telechargement/download/...
162,93,2021,https://data.geopf.fr/telechargement/download/...


In [134]:
file = liste_files[0]

In [135]:
liste_dep.loc[liste_dep.codgeo==file[9:11], 'year'].values

array([2021], dtype=int64)

In [136]:
temp = []
for file in liste_files:
    df_temp = pd.read_csv('carreaux/'+ file)
    df_temp['annee'] = liste_dep.loc[liste_dep.codgeo==file[9:11], 'year'].values[0]
    df_temp['codgeo'] = file[9:11]
    temp.append(df_temp)

In [137]:
temp_annee = pd.concat(temp).groupby(['x', 'y', 'annee'], as_index=False).surface.sum()
temp_annee['surface_annee'] = temp_annee.annee * temp_annee.surface
temp_annee = temp_annee.groupby(['x', 'y'], as_index=False)[['surface', 'surface_annee']].sum()
temp_annee['annee'] = temp_annee.surface_annee / temp_annee.surface
temp_annee['surface'] = temp_annee.surface / size**2
carreaux = temp_annee[['x', 'y', 'annee', 'surface']]

In [138]:
temp_cs = pd.concat(temp).groupby(['x', 'y', 'code_cs'], as_index=False).surface.sum()
temp_cs.surface = temp_cs.surface / size**2 
carreaux_cs = temp_cs.pivot(values='surface', index=['x', 'y'], columns='code_cs').reset_index().fillna(0)

In [139]:
temp_us = pd.concat(temp).groupby(['x', 'y', 'code_us'], as_index=False).surface.sum()
temp_us.surface = temp_us.surface / size**2 
carreaux_us = temp_us.pivot(values='surface', index=['x', 'y'], columns='code_us').reset_index().fillna(0)

In [140]:
temp_cs_us = pd.concat(temp).groupby(['x', 'y', 'code_cs', 'code_us'], as_index=False).surface.sum()
temp_cs_us.surface = temp_cs_us.surface / size**2 
carreaux_cs_us = temp_cs_us.pivot(values='surface', index=['x', 'y'], columns=['code_cs', 'code_us']).reset_index().fillna(0)

In [141]:
columns =  ['_'.join(c)   for c in  carreaux_cs_us.columns]
columns[0] = 'x'
columns[1] = 'y'
carreaux_cs_us.columns = columns

In [142]:
carreaux = pd.merge(carreaux, carreaux_cs, left_on=['x', 'y'], right_on=['x', 'y'])  
carreaux = pd.merge(carreaux, carreaux_us, left_on=['x', 'y'], right_on=['x', 'y'])
carreaux = pd.merge(carreaux, carreaux_cs_us, left_on=['x', 'y'], right_on=['x', 'y'])

In [143]:
geometry = [Polygon([(x,y),(x+size,y),(x+size,y+size),(x,y+size),(x,y) ]) for x,y in zip(carreaux.y, carreaux.x)]

In [144]:
carreaux = gpd.GeoDataFrame(carreaux, geometry=geometry, crs=3035)

In [145]:
carreaux['id_carreau_1km'] =  "FR_CRS3035RES" + str(size) + "mN" + carreaux.x.astype(str) + "E" + carreaux.y.astype(str) 

In [146]:
carreaux.columns = [col.replace('.', '_') for col in carreaux.columns]

In [147]:
carreaux['CS1_1'] = carreaux['CS1_1_1_1'] + carreaux['CS1_1_1_2'] + carreaux['CS1_1_2_1'] + carreaux['CS1_1_2_2']
carreaux['CS1_2'] = carreaux['CS1_2_1'] + carreaux['CS1_2_2'] + carreaux['CS1_2_3']
carreaux['CS2_1'] = carreaux['CS2_1_1_1'] + carreaux['CS2_1_1_2'] + carreaux['CS2_1_1_3'] + carreaux['CS2_1_2'] + carreaux['CS2_1_3']
carreaux['CS2_2'] = carreaux['CS2_2_1'] #+ carreaux['CS2_2_2']
carreaux['CS1'] = carreaux['CS1_1'] + carreaux['CS1_2']
carreaux['CS2'] = carreaux['CS2_1'] + carreaux['CS2_2']

In [148]:
carreaux.drop(columns=['x', 'y'], inplace=True)

In [149]:
carreaux = carreaux[['id_carreau_1km', 'annee',  'surface', 'CS1_1', 'CS1_2', 'CS2_1', 'CS2_2', 'CS1', 'CS2',
                     'CS1_1_1_1',  'CS1_1_1_2',  'CS1_1_2_1',
                     'CS1_1_2_2', 'CS1_2_1', 'CS1_2_2', 'CS1_2_3', 'CS2_1_1_1', 'CS2_1_1_2', 'CS2_1_1_3',
                     'CS2_1_2', 'CS2_1_3', 'CS2_2_1', 'US1_1', 'US1_2', 'US1_3', 'US1_4', 'US1_5', 'US2', 
                     'US235', 'US3', 'US4_1_1', 'US4_1_2', 'US4_1_3', 'US4_1_4', 'US4_1_5', 'US4_2', 'US4_3',
                     'US5', 'US6_1', 'US6_2', 'US6_3', 'US6_6', 'CS1_2_1_US6_3', 'CS1_2_2_US6_3', 'CS1_1_1_1_US3', 
                     'CS1_1_1_2_US3', 'CS2_1_2_US6_3', 'CS2_2_1_US3', 'CS2_2_1_US6_3', 'CS1_2_2_US4_1_4', 'CS1_1_1_1_US6_1',
                     'CS2_1_1_3_US1_2', 'CS1_1_1_1_US5', 'CS2_1_1_1_US1_2', 'CS2_1_1_1_US5', 'CS2_2_1_US5', 'CS1_1_1_2_US4_1_1',
                     'CS1_1_2_1_US5', 'CS2_1_1_3_US3', 'CS1_1_2_1_US3', 'CS1_1_2_1_US4_1_1', 'CS2_1_1_1_US3', 'CS1_1_2_1_US4_1_4',
                     'CS2_2_1_US4_1_1', 'CS2_1_1_2_US1_2', 'CS2_1_2_US1_1', 'CS1_1_1_1_US1_1', 'CS2_1_2_US3', 'CS2_2_1_US1_1',
                     'CS1_1_1_2_US5', 'CS2_1_1_1_US1_1', 'CS2_1_1_3_US5', 'CS2_1_2_US5', 'CS1_1_2_1_US4_3', 'CS1_1_2_1_US6_1',
                     'CS1_2_2_US3', 'CS2_1_1_1_US6_3', 'CS2_1_1_2_US5', 'CS1_1_1_2_US1_1', 'CS1_1_1_1_US4_3', 'CS2_2_1_US2',
                     'CS2_2_1_US4_3', 'CS2_1_1_3_US6_3', 'CS2_2_1_US6_1', 'CS1_1_2_1_US1_1', 'CS2_1_3_US1_1', 'CS1_1_1_1_US2',
                     'CS1_1_1_1_US6_2', 'CS1_1_2_1_US2', 'CS2_1_2_US2', 'CS1_1_2_1_US235', 'CS1_1_1_2_US4_3', 'CS2_1_1_2_US3',
                     'CS2_1_2_US235', 'CS2_1_1_2_US6_3', 'CS2_1_2_US1_2', 'CS1_1_2_1_US1_3', 'CS2_1_2_US1_3', 'CS2_1_1_1_US4_1_1',
                     'CS1_1_1_1_US1_3', 'CS2_1_1_1_US1_3', 'CS1_1_1_2_US4_1_3', 'CS2_2_1_US4_1_3', 'CS1_1_1_1_US4_1_3', 'CS1_1_1_2_US2',
                     'CS1_1_2_1_US4_1_3', 'CS2_1_2_US4_1_3', 'CS2_1_2_US4_3', 'CS2_1_2_US4_1_1', 'CS1_1_1_1_US235', 'CS1_1_1_2_US1_3',
                     'CS1_2_2_US1_3', 'CS2_2_1_US1_3', 'CS1_1_1_2_US235', 'CS1_2_2_US4_3', 'CS2_1_1_1_US235', 'CS2_2_1_US235',
                     'CS1_1_1_1_US4_2', 'CS1_1_1_2_US4_2', 'CS2_1_1_2_US4_1_1', 'CS2_1_1_1_US2', 'CS1_1_1_2_US4_1_4', 'CS1_1_1_1_US4_1_4',
                     'CS1_1_2_2_US2', 'CS1_1_2_1_US6_2', 'CS1_1_1_2_US4_1_2', 'CS1_1_1_1_US4_1_2', 'CS1_1_2_1_US4_1_2', 'CS2_2_1_US4_1_2',
                     'CS2_1_2_US4_1_2', 'CS2_1_2_US6_1', 'CS2_1_1_3_US4_1_1', 'CS1_1_1_2_US6_1', 'CS1_1_1_1_US4_1_1', 'CS1_2_2_US1_1',
                     'CS2_1_1_3_US2', 'CS1_1_2_1_US4_2', 'CS2_1_1_2_US2', 'CS1_1_1_1_US1_4', 'CS1_1_1_2_US1_4', 'CS2_2_1_US1_4',
                     'CS1_2_2_US5', 'CS1_2_2_US2', 'CS2_1_1_2_US235', 'CS2_1_3_US5', 'CS1_1_1_2_US6_2', 'CS2_2_1_US4_2',
                     'CS1_1_1_1_US4_1_5', 'CS1_1_2_1_US4_1_5', 'CS1_2_3_US6_3', 'CS1_1_2_1_US6_3', 'CS1_1_2_1_US1_4', 'CS1_2_2_US1_4',
                     'CS2_1_2_US6_2', 'CS2_1_1_3_US235', 'CS2_1_3_US3', 'CS1_1_2_1_US1_2', 'CS2_1_1_3_US1_3', 'CS2_1_1_2_US1_3',
                     'CS2_1_1_1_US6_2', 'CS2_2_1_US4_1_5', 'CS1_2_2_US235', 'CS1_1_2_2_US4_3', 'CS1_1_2_2_US235', 'CS2_2_1_US6_6',
                     'CS1_1_2_2_US3', 'CS1_1_2_1_US6_6', 'CS2_1_2_US4_2', 'CS1_2_2_US4_1_1', 'CS1_1_1_2_US1_2', 'CS2_1_3_US235',
                     'CS1_1_2_2_US5', 'CS1_1_1_1_US6_6', 'CS1_1_1_2_US6_6', 'CS2_1_1_1_US6_6', 'CS2_1_1_1_US1_5', 'CS2_1_1_1_US4_3',
                     'CS2_1_2_US1_5', 'CS1_1_2_2_US6_2', 'CS2_1_1_1_US4_1_2', 'CS2_1_1_2_US4_3', 'CS2_2_1_US4_1_4', 'CS2_1_1_1_US4_1_3',
                     'CS2_1_1_3_US4_1_2', 'CS2_1_1_2_US4_1_2', 'CS2_1_3_US2', 'CS2_2_1_US6_2',  'CS2_1_1_3_US6_2', 'CS2_1_1_2_US4_1_3',
                     'CS1_1_1_2_US4_1_5', 'CS1_1_1_2_US6_3',  'geometry']]

In [150]:
carreaux.to_parquet('ocsge_2022_carreaux_1000m.parquet')
gpio.read('ocsge_carreaux_1000m.parquet') \
    .add_bbox() \
    .sort_hilbert() \
    .write('ocsge_2022_carreaux_1000m.parquet', compression='ZSTD')

WindowsPath('ocsge_2022_carreaux_1000m.parquet')

## Département

In [151]:
temp_annee = pd.concat(temp).groupby(['codgeo', 'annee'], as_index=False).surface.sum()
temp_annee['surface_annee'] = temp_annee.annee * temp_annee.surface
temp_annee = temp_annee.groupby(['codgeo'], as_index=False)[['surface', 'surface_annee']].sum()
temp_annee['annee'] = temp_annee.surface_annee / temp_annee.surface
temp_annee['surface'] = temp_annee.surface / size**2
departement = temp_annee[['codgeo', 'annee', 'surface']]

In [152]:
temp_cs = pd.concat(temp).groupby(['codgeo', 'code_cs'], as_index=False).surface.sum()
temp_cs.surface = temp_cs.surface / size**2 
departement_cs = temp_cs.pivot(values='surface', index=['codgeo'], columns='code_cs').reset_index().fillna(0)

In [153]:
temp_us = pd.concat(temp).groupby(['codgeo', 'code_us'], as_index=False).surface.sum()
temp_us.surface = temp_us.surface / size**2 
departement_us = temp_us.pivot(values='surface', index=['codgeo'], columns='code_us').reset_index().fillna(0)

In [154]:
temp_cs_us = pd.concat(temp).groupby(['codgeo', 'code_cs', 'code_us'], as_index=False).surface.sum()
temp_cs_us.surface = temp_cs_us.surface / size**2 
departement_cs_us = temp_cs_us.pivot(values='surface', index=['codgeo'], columns=['code_cs', 'code_us']).reset_index().fillna(0)

columns =  ['_'.join(c)   for c in  departement_cs_us.columns]
columns[0] = 'codgeo'
departement_cs_us.columns = columns

In [155]:
departement = pd.merge(departement, departement_cs, left_on=['codgeo'], right_on=['codgeo'])  
departement = pd.merge(departement, departement_us, left_on=['codgeo'], right_on=['codgeo'])
departement = pd.merge(departement, departement_cs_us, left_on=['codgeo'], right_on=['codgeo'])

In [156]:
departement.columns = [col.replace('.', '_') for col in departement.columns]

In [157]:
departement['CS1_1'] = departement['CS1_1_1_1'] + departement['CS1_1_1_2'] + departement['CS1_1_2_1'] + departement['CS1_1_2_2']
departement['CS1_2'] = departement['CS1_2_1'] + departement['CS1_2_2'] + departement['CS1_2_3']
departement['CS2_1'] = departement['CS2_1_1_1'] + departement['CS2_1_1_2'] + departement['CS2_1_1_3'] + departement['CS2_1_2'] + departement['CS2_1_3']
departement['CS2_2'] = departement['CS2_2_1'] #+ carreaux['CS2_2_2']
departement['CS1'] = departement['CS1_1'] + departement['CS1_2']
departement['CS2'] = departement['CS2_1'] + departement['CS2_2']

In [158]:
departement = departement[['codgeo', 'annee',  'surface', 'CS1_1', 'CS1_2', 'CS2_1', 'CS2_2', 'CS1', 'CS2',
                     'CS1_1_1_1',  'CS1_1_1_2',  'CS1_1_2_1',
                     'CS1_1_2_2', 'CS1_2_1', 'CS1_2_2', 'CS1_2_3', 'CS2_1_1_1', 'CS2_1_1_2', 'CS2_1_1_3',
                     'CS2_1_2', 'CS2_1_3', 'CS2_2_1', 'US1_1', 'US1_2', 'US1_3', 'US1_4', 'US1_5', 'US2', 
                     'US235', 'US3', 'US4_1_1', 'US4_1_2', 'US4_1_3', 'US4_1_4', 'US4_1_5', 'US4_2', 'US4_3',
                     'US5', 'US6_1', 'US6_2', 'US6_3', 'US6_6', 'CS1_2_1_US6_3', 'CS1_2_2_US6_3', 'CS1_1_1_1_US3', 
                     'CS1_1_1_2_US3', 'CS2_1_2_US6_3', 'CS2_2_1_US3', 'CS2_2_1_US6_3', 'CS1_2_2_US4_1_4', 'CS1_1_1_1_US6_1',
                     'CS2_1_1_3_US1_2', 'CS1_1_1_1_US5', 'CS2_1_1_1_US1_2', 'CS2_1_1_1_US5', 'CS2_2_1_US5', 'CS1_1_1_2_US4_1_1',
                     'CS1_1_2_1_US5', 'CS2_1_1_3_US3', 'CS1_1_2_1_US3', 'CS1_1_2_1_US4_1_1', 'CS2_1_1_1_US3', 'CS1_1_2_1_US4_1_4',
                     'CS2_2_1_US4_1_1', 'CS2_1_1_2_US1_2', 'CS2_1_2_US1_1', 'CS1_1_1_1_US1_1', 'CS2_1_2_US3', 'CS2_2_1_US1_1',
                     'CS1_1_1_2_US5', 'CS2_1_1_1_US1_1', 'CS2_1_1_3_US5', 'CS2_1_2_US5', 'CS1_1_2_1_US4_3', 'CS1_1_2_1_US6_1',
                     'CS1_2_2_US3', 'CS2_1_1_1_US6_3', 'CS2_1_1_2_US5', 'CS1_1_1_2_US1_1', 'CS1_1_1_1_US4_3', 'CS2_2_1_US2',
                     'CS2_2_1_US4_3', 'CS2_1_1_3_US6_3', 'CS2_2_1_US6_1', 'CS1_1_2_1_US1_1', 'CS2_1_3_US1_1', 'CS1_1_1_1_US2',
                     'CS1_1_1_1_US6_2', 'CS1_1_2_1_US2', 'CS2_1_2_US2', 'CS1_1_2_1_US235', 'CS1_1_1_2_US4_3', 'CS2_1_1_2_US3',
                     'CS2_1_2_US235', 'CS2_1_1_2_US6_3', 'CS2_1_2_US1_2', 'CS1_1_2_1_US1_3', 'CS2_1_2_US1_3', 'CS2_1_1_1_US4_1_1',
                     'CS1_1_1_1_US1_3', 'CS2_1_1_1_US1_3', 'CS1_1_1_2_US4_1_3', 'CS2_2_1_US4_1_3', 'CS1_1_1_1_US4_1_3', 'CS1_1_1_2_US2',
                     'CS1_1_2_1_US4_1_3', 'CS2_1_2_US4_1_3', 'CS2_1_2_US4_3', 'CS2_1_2_US4_1_1', 'CS1_1_1_1_US235', 'CS1_1_1_2_US1_3',
                     'CS1_2_2_US1_3', 'CS2_2_1_US1_3', 'CS1_1_1_2_US235', 'CS1_2_2_US4_3', 'CS2_1_1_1_US235', 'CS2_2_1_US235',
                     'CS1_1_1_1_US4_2', 'CS1_1_1_2_US4_2', 'CS2_1_1_2_US4_1_1', 'CS2_1_1_1_US2', 'CS1_1_1_2_US4_1_4', 'CS1_1_1_1_US4_1_4',
                     'CS1_1_2_2_US2', 'CS1_1_2_1_US6_2', 'CS1_1_1_2_US4_1_2', 'CS1_1_1_1_US4_1_2', 'CS1_1_2_1_US4_1_2', 'CS2_2_1_US4_1_2',
                     'CS2_1_2_US4_1_2', 'CS2_1_2_US6_1', 'CS2_1_1_3_US4_1_1', 'CS1_1_1_2_US6_1', 'CS1_1_1_1_US4_1_1', 'CS1_2_2_US1_1',
                     'CS2_1_1_3_US2', 'CS1_1_2_1_US4_2', 'CS2_1_1_2_US2', 'CS1_1_1_1_US1_4', 'CS1_1_1_2_US1_4', 'CS2_2_1_US1_4',
                     'CS1_2_2_US5', 'CS1_2_2_US2', 'CS2_1_1_2_US235', 'CS2_1_3_US5', 'CS1_1_1_2_US6_2', 'CS2_2_1_US4_2',
                     'CS1_1_1_1_US4_1_5', 'CS1_1_2_1_US4_1_5', 'CS1_2_3_US6_3', 'CS1_1_2_1_US6_3', 'CS1_1_2_1_US1_4', 'CS1_2_2_US1_4',
                     'CS2_1_2_US6_2', 'CS2_1_1_3_US235', 'CS2_1_3_US3', 'CS1_1_2_1_US1_2', 'CS2_1_1_3_US1_3', 'CS2_1_1_2_US1_3',
                     'CS2_1_1_1_US6_2', 'CS2_2_1_US4_1_5', 'CS1_2_2_US235', 'CS1_1_2_2_US4_3', 'CS1_1_2_2_US235', 'CS2_2_1_US6_6',
                     'CS1_1_2_2_US3', 'CS1_1_2_1_US6_6', 'CS2_1_2_US4_2', 'CS1_2_2_US4_1_1', 'CS1_1_1_2_US1_2', 'CS2_1_3_US235',
                     'CS1_1_2_2_US5', 'CS1_1_1_1_US6_6', 'CS1_1_1_2_US6_6', 'CS2_1_1_1_US6_6', 'CS2_1_1_1_US1_5', 'CS2_1_1_1_US4_3',
                     'CS2_1_2_US1_5', 'CS1_1_2_2_US6_2', 'CS2_1_1_1_US4_1_2', 'CS2_1_1_2_US4_3', 'CS2_2_1_US4_1_4', 'CS2_1_1_1_US4_1_3',
                     'CS2_1_1_3_US4_1_2', 'CS2_1_1_2_US4_1_2', 'CS2_1_3_US2', 'CS2_2_1_US6_2',  'CS2_1_1_3_US6_2', 'CS2_1_1_2_US4_1_3',
                     'CS1_1_1_2_US4_1_5', 'CS1_1_1_2_US6_3']]

In [176]:
departement['annee'] = np.floor(departement.annee)

In [180]:
departement.to_csv('ocsge_2022_departement_1000m.csv', index=False)

## Verification

In [165]:
print(carreaux.surface.sum())
print(carreaux.CS1.sum() + carreaux.CS2.sum())

549146.6715599642
549146.6715599641


In [166]:
print(departement.surface.sum())
print(departement.CS1.sum() + departement.CS2.sum())

549146.6715599641
549146.6715599642


In [179]:
np.sum(departement.annee * departement.surface) / departement.surface.sum()

2021.6700623253914